# Predict

In [1]:
import os
import pandas as pd
import torch
import evaluate

from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

/home/psaegert/miniconda3/envs/fsem/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from nlinec import get_positive_data, get_all_types, get_results_dir, get_models_dir, combine_premise_hypothesis
from nlinec.predict import predict_probabilities

## Parameters

In [3]:
# Specify the dataset to predict and a file to save the predictions to
SPLIT = "dev"
DATASET = f'g_{SPLIT}.json'
MODEL = "nlinec-E-2"
HYPOTHESIS_ONLY = False

SAVE_MODEL_TO = os.path.join(get_models_dir(), MODEL)
SAVE_PREDICTIONS_TO = os.path.join(get_results_dir(), MODEL, f"{SPLIT}_predictions" + ("_ho" if HYPOTHESIS_ONLY else "") + ".csv")

# Specify the parameters for the prediction
SAVE_EVERY = 100_000

# Use the GPU if available
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Make sure the directory to save the predictions to exists
os.makedirs(os.path.dirname(SAVE_PREDICTIONS_TO), exist_ok=True)

## Load Models & Data

In [4]:
tokenizer = AutoTokenizer.from_pretrained('roberta-large-mnli')

if MODEL == 'roberta-large-mnli':
    model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(DEVICE)
else:
    model = AutoModelForSequenceClassification.from_pretrained(SAVE_MODEL_TO).to(DEVICE)

In [5]:
# Make entailment predictions for all types and filter out the relevant ones later in the analysis
all_types = get_all_types(granularity=-1)
all_types

,full_type,type
0,/other,other
1,/location/country,country
2,/location,location
3,/other/scientific,scientific
4,/location/city,city
...,...,...
84,/location/geography/body_of_water,body_of_water
85,/location/geograpy/island,island
86,/location/geograpy,geograpy
87,/other/legal,legal


In [6]:
# Load the dataset
data = get_positive_data(DATASET)
data

Loading g_dev.json: 2202it [00:00, 221829.17it/s]


,mention_span,full_type,sentence,label
0,Friday,[/other],Japan's wholesale prices in September rose 3.3...,2
1,September,[/other],Japan's wholesale prices in September rose 3.3...,2
2,Japan,"[/location, /location/country]",Japan's wholesale prices in September rose 3.3...,2
3,the Bank of Japan,"[/location, /location/structure, /organization...",Japan's wholesale prices in September rose 3.3...,2
4,3.3 %,[/other],Japan's wholesale prices in September rose 3.3...,2
...,...,...,...,...
2197,the Treasury 's,"[/organization, /organization/government]","The non-callable issue, which can be put back ...",2
2198,$ 500 million of Remic mortgage securities,[/other],$ 500 million of Remic mortgage securities of...,2
2199,the Treasury 's,"[/organization, /organization/government]","The issue, which is puttable back to the compa...",2
2200,200 basis points,[/other],Among classes for which details were available...,2


In [7]:
# If some predictions already exist, load them
if os.path.exists(SAVE_PREDICTIONS_TO):
    # Load the predictions from file
    print("Loading predictions from file")
    predictions_df = pd.read_csv(SAVE_PREDICTIONS_TO, index_col=0)
else:
    # Create a dataframe with the same index as the data
    predictions_df = pd.DataFrame(columns=list(all_types['full_type']), index=data.index)

In [9]:
# Find out which predictions still need to be made
todo = predictions_df.isna().any(axis=1)
print(f'Progress: {(~todo).mean() * 100:.2%}')

Progress: 0.00%


## Predict

### NEC

In [10]:
# Make predictions for the remaining rows
with torch.no_grad():  # Disable gradient calculation for speed
    # Keep track of how many predictions have been made since the last save
    new_predictions_counter = 0

    # Iterate over all rows in the dev data
    for row in tqdm(data.loc[todo, :].itertuples(), total=todo.sum()):

        # Predict the type of the mention and store the prediction
        entailment_probabilities = predict_probabilities(
            model,
            tokenizer,
            row.sentence,
            row.mention_span,
            all_types['type'],
            hypothesis_only=HYPOTHESIS_ONLY)[0, :, -1]  # -1 is the entailment class

        # Store the prediction
        predictions_df.loc[row.Index, :] = entailment_probabilities

        # Save the predictions to file every SAVE_EVERY predictions
        new_predictions_counter += 1
        if new_predictions_counter >= SAVE_EVERY:
            
            # Save the predictions to file
            predictions_df.to_csv(SAVE_PREDICTIONS_TO)
            new_predictions_counter = 0

# Save the remaining predictions to file
predictions_df.to_csv(SAVE_PREDICTIONS_TO)

100%|██████████| 2202/2202 [03:37<00:00, 10.12it/s]


### Multi-NLI

In [ ]:
dataset = load_dataset("multi_nli", split='validation_matched')
metric = evaluate.load('accuracy')

In [ ]:
model.eval()
with torch.no_grad():
    predictions = []
    for instance in tqdm(dataset):
        input_text = combine_premise_hypothesis(instance["premise"], instance["hypothesis"])
        tokenized_input_text = tokenizer(input_text, max_length=model.config.max_position_embeddings, padding="max_length", return_tensors='pt')
        outputs = model(tokenized_input_text['input_ids'].to(DEVICE), tokenized_input_text['attention_mask'].to(DEVICE))
        predictions.append(outputs[0].argmax(dim=1).cpu().numpy()[0])

In [ ]:
# This took an embarrassingly long time to figure out. Why is the model's label mapping different from the dataset's label mapping?
# https://huggingface.co/datasets/multi_nli/viewer/default/validation_matched
dataset_id2label = {'ENTAILMENT': 0, 'NEUTRAL': 1, 'CONTRADICTION': 2}
model_id2label = model.config.id2label

# Convert the model's predictions to the dataset's labels
model_labels_for_dataset = [dataset_id2label[model_id2label[logit]] for logit in predictions]

# Compute the accuracy
accuracy = metric.compute(references=dataset['label'], predictions=model_labels_for_dataset)

print(f'Accuracy on MultiNLI: {accuracy["accuracy"]:.3f}')

# Save the accuracy
pd.DataFrame(accuracy, index=['accuracy']).rename(columns={'accuracy': MODEL}).to_csv(os.path.join(get_results_dir(), f'{MODEL}', 'mnli_accuracy.csv'))

Accuracy on MultiNLI: 0.701
